In [1]:
import polars as pl
import pandas as pd
import plotly.express as px 
import pyarrow
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [2]:
lf = pl.scan_csv('D:\Base_de_dados_anvisa\Top10_substances.csv')

lf.head().collect()

PRINCIPIO_ATIVO,UF_VENDA,DATA,count
str,str,i64,i64
"""ESTOLATO DE ER...","""SP""",102014,5948
"""CLORIDRATO DE ...","""SP""",102014,5885
"""ESTOLATO DE ER...","""SP""",72014,5809
"""CLORIDRATO DE ...","""SP""",92014,5696
"""CLORIDRATO DE ...","""SP""",72014,5618


In [3]:
lf_hp = pl.scan_csv('D:\Base_de_dados_anvisa\hipertrofia.csv')

lf_hp.head().collect()

Mês,Hipertrofia: (Brasil)
str,i64
"""2014-01""",97
"""2014-02""",98
"""2014-03""",84
"""2014-04""",80
"""2014-05""",82


In [4]:
lf_gt = pl.scan_csv('D:\Base_de_dados_anvisa\multiTimeline.csv')

lf_gt.head().collect()

Mês,impotência: (Brasil)
str,i64
"""2014-01""",19
"""2014-02""",18
"""2014-03""",20
"""2014-04""",20
"""2014-05""",22


In [5]:
def correct_data(LazyFrame, col_name):
    z=8
    q = (
        lf
        .with_columns(
            pl.col(col_name)
            .cast(pl.Utf8)
        )
        .with_columns(
            pl.when(
                pl.col(col_name).cast(pl.Utf8).str.lengths() > z
            )
            .then(
                pl.col(col_name).cast(pl.Utf8)
            )
            .otherwise(
                pl.concat_str([pl.lit("0" * z), pl.col(col_name).cast(pl.Utf8)]).str.slice(-z)
            )
            .alias("DATETIME")
        )
        .with_columns(
            pl.col('DATETIME').str.replace('00','01').alias('DATETIME')
        )
        .with_columns(
            pl.col('DATETIME').str.strptime(pl.Date, fmt='%d%m%Y').cast(pl.Datetime)
        )
        .filter(
            pl.col('DATETIME') <= pl.lit((datetime(2021, 7, 1)))
        )
        .sort(
            by=pl.col('DATETIME')
        )

    )
    df = q.collect()
    
    return df


In [6]:
# Convert str to datetime from google trends csv
def googleLFDataPrep(lf):   
    lf_gt =(
             lf
            .with_columns(
                pl.col('Mês').str.strptime(pl.Date, fmt='%Y-%m').cast(pl.Datetime)
            )
    ).collect()
    
    return lf_gt

In [7]:
df = correct_data(lf, 'DATA')
df

PRINCIPIO_ATIVO,UF_VENDA,DATA,count,DATETIME
str,str,str,i64,datetime[μs]
"""CLORIDRATO DE ...","""SP""","""12014""",5055,2014-01-01 00:00:00
"""CLORIDRATO DE ...","""SP""","""12014""",4869,2014-01-01 00:00:00
"""ESTOLATO DE ER...","""SP""","""12014""",4218,2014-01-01 00:00:00
"""CLORIDRATO DE ...","""SP""","""12014""",3976,2014-01-01 00:00:00
"""TOPIRAMATO""","""SP""","""12014""",3643,2014-01-01 00:00:00
"""CLORIDRATO DE ...","""SP""","""12014""",3443,2014-01-01 00:00:00
"""CLORIDRATO DE ...","""SP""","""12014""",3177,2014-01-01 00:00:00
"""CLORIDRATO DE ...","""SP""","""12014""",2365,2014-01-01 00:00:00
"""GABAPENTINA""","""SP""","""12014""",1841,2014-01-01 00:00:00


## Avaliando as substâncias mais vendidas

In [8]:
def TopSubstances(df, every_period='1mo'): 
    df_substances = (
        df
        .groupby(
            ['PRINCIPIO_ATIVO','DATETIME']).agg(pl.col('count').sum()
        )
        .sort(
            'DATETIME'
            )
        .groupby_dynamic(
            index_column='DATETIME',by='PRINCIPIO_ATIVO', every=every_period, period=every_period
            )
            .agg(
                [pl.col('count').sum()]
                )
        .filter(
            pl.struct('PRINCIPIO_ATIVO').is_in([
                                                {'PRINCIPIO_ATIVO':'GABAPENTINA'}, 
                                                {'PRINCIPIO_ATIVO':'TESTOSTERONA'},
                                                {'PRINCIPIO_ATIVO':'CLORIDRATO DE AMITRIPTILINA'},
                                                {'PRINCIPIO_ATIVO':'CLORIDRATO DE BUPROPIONA'},
                                                {'PRINCIPIO_ATIVO':'TOPIRAMATO'},
                                                {'PRINCIPIO_ATIVO':'ESTOLATO DE ERITROMICINA'},
                                                {'PRINCIPIO_ATIVO':'CLORIDRATO DE SERTRALINA'},
                                                {'PRINCIPIO_ATIVO':'CLORIDRATO DE PAROXETINA'},
                                                {'PRINCIPIO_ATIVO':'CLORIDRATO DE TRAMADOL'},
                                                {'PRINCIPIO_ATIVO':'CLORIDRATO DE FLUOXETINA'}
                                                ])
        )
    )
    return df_substances


In [9]:
df_substances = TopSubstances(df,'1mo')
df_substances = df_substances.to_pandas()
px.line(df_substances, x='DATETIME', y='count', color='PRINCIPIO_ATIVO').update_layout(
                                                                            xaxis_title="Data", yaxis_title="Volume de vendas"
                                                                         )

In [112]:
df_variation = TopSubstances(df,'1mo')
df_variation = (
    df_variation
    .filter(
            (
            pl.col('DATETIME')<=datetime(2014,1,1)
            )|
            (
            pl.col('DATETIME')==datetime(2021,7,1)
            )
    )
    .groupby('PRINCIPIO_ATIVO').agg(
                                    [
                                    pl.col('count').min().alias('initial_date'),
                                    pl.col('count').max().alias('final_date')
                                    ]
                                )
    .with_columns(
        (pl.col('final_date')/pl.col('initial_date'))
        .alias('variation')
    )
    .sort(by='variation', reverse=True)

)
df_variation


PRINCIPIO_ATIVO,initial_date,final_date,variation
str,i64,i64,f64
"""TESTOSTERONA""",4293,18917,4.406476
"""GABAPENTINA""",4551,17227,3.785322
"""CLORIDRATO DE ...",5760,10681,1.85434
"""TOPIRAMATO""",8150,12159,1.491902
"""CLORIDRATO DE ...",8794,12957,1.473391
"""CLORIDRATO DE ...",8933,10950,1.225792
"""CLORIDRATO DE ...",11754,13320,1.133231
"""CLORIDRATO DE ...",11725,13212,1.126823
"""CLORIDRATO DE ...",8577,9181,1.070421


In [113]:
df_substances = TopSubstances(df,'1mo')

df_subs_totals = df_substances.groupby('PRINCIPIO_ATIVO').agg([pl.col('count').sum()]).sort(by='count', reverse=True)
df_subs_totals = df_subs_totals.to_pandas()

df_variation = df_variation.to_pandas()


In [144]:
fig = make_subplots(specs=[[{"secondary_y": True}]],shared_xaxes=True)

fig.add_trace(go.Bar(
    x=df_variation['PRINCIPIO_ATIVO'],
    y=df_variation['variation'],
    yaxis='y',
    name='Crescimento (2014→2021)',
    ))
fig.add_trace(go.Bar(
    x=df_subs_totals['PRINCIPIO_ATIVO'],
    y=df_subs_totals['count'],
    yaxis='y2',
    name='Vendas totais no períoro (2014-2021)',
    width=0.4,
    opacity=0.2
    )
    )

fig.update_layout(
width=1700,
height=1000,
legend=dict(
        x=0.57,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=25,
            color="black"
        )),
xaxis=dict(
    # title='Data',
    titlefont=dict(
        size=30
    ),
    tickfont=dict(
        size=25
    )
),

yaxis=dict(
    title="<b>Variação (9 anos)<b>",
    titlefont=dict(
        color="blue",
        size=30
        
    ),
    tickformat = "2%",

    tickfont=dict(
        color="blue",
        size=25
    )
),
yaxis2=dict(
    title="<b>Vendas totais no período (2014-2021)",
    titlefont=dict(
        color="red",
        size=30
    ),
    tickfont=dict(
        color="red",
        size=25
    ),
    #anchor="free",
    overlaying="y",
    side="right",
    range=[0,1500000]
    # position=0.15
))




In [63]:
df_substances = TopSubstances(df,'1mo')



df_crescimento = (df_substances
            .groupby_dynamic(
                index_column='DATETIME',by='PRINCIPIO_ATIVO', every='1mo', period='1mo'
            )
            .agg(
                [pl.col('count').mean()]
            )
            .filter(
                (
                    pl.col('DATETIME')<=datetime(2014,1,1)
                    )|
                (
                    pl.col('DATETIME')==datetime(2021,7,1)
                    )
                )
            .groupby(
                'PRINCIPIO_ATIVO'
                )
                .agg(
                    pl.col('count')
                    )
            )

df_crescimento_f = (df_substances
            .groupby_dynamic(
                index_column='DATETIME',by='PRINCIPIO_ATIVO', every='1mo', period='1mo'
            )
            .agg(
                [pl.col('count').mean()]
            )
            .filter(
                # (
                #     pl.col('DATETIME')<=datetime(2014,1,1)
                #     )|
                (
                    pl.col('DATETIME')==datetime(2021,7,1)
                    )
                )
            .groupby(
                'PRINCIPIO_ATIVO'
                )
                .agg(
                    pl.col('count')
                    )
            )

df_crescimento_l = (df_substances
            .groupby_dynamic(
                index_column='DATETIME',by='PRINCIPIO_ATIVO', every='1mo', period='1mo'
            )
            .agg(
                [pl.col('count').mean()]
            )
            .filter(
                (
                    pl.col('DATETIME')<=datetime(2014,1,1)
                    )
                # (
                #     pl.col('DATETIME')==datetime(2021,7,1)
                #     )
                )
            .groupby(
                'PRINCIPIO_ATIVO'
                )
                .agg(
                    pl.col('count')
                    )
            )

lista1 = df_crescimento_f['count'].to_list()
lista2 = df_crescimento_l['count'].to_list()


In [66]:
lista1[0:]

[[12957.0],
 [17227.0],
 [10681.0],
 [8577.0],
 [11725.0],
 [18917.0],
 [10975.0],
 [8933.0],
 [12159.0],
 [13320.0]]

In [67]:
df_teste = df_crescimento.with_columns(pl.Series(name='1', values=lista1))
df_teste

PRINCIPIO_ATIVO,count,1
str,list[f64],list[f64]
"""CLORIDRATO DE ...","[10950.0, 8933.0]",[12957.0]
"""CLORIDRATO DE ...","[11754.0, 13320.0]",[17227.0]
"""TOPIRAMATO""","[8150.0, 12159.0]",[10681.0]
"""CLORIDRATO DE ...","[5760.0, 10681.0]",[8577.0]
"""TESTOSTERONA""","[4293.0, 18917.0]",[11725.0]
"""CLORIDRATO DE ...","[8794.0, 12957.0]",[18917.0]
"""CLORIDRATO DE ...","[13212.0, 11725.0]",[10975.0]
"""ESTOLATO DE ER...","[10273.0, 10975.0]",[8933.0]
"""CLORIDRATO DE ...","[9181.0, 8577.0]",[12159.0]


## Estudo sobre testosterona

In [13]:
df_substances = TopSubstances(df,'1mo')
df_substances = df_substances.to_pandas()

lf_gt = googleLFDataPrep(lf_gt)
lf_gt = lf_gt.to_pandas()


lf_hp = googleLFDataPrep(lf_hp)
lf_hp = lf_hp.to_pandas()

In [14]:
fig = make_subplots(specs=[[{"secondary_y": True}]],shared_xaxes=True,)

fig.add_trace(go.Scatter(
                        x=df_substances['DATETIME'],
                        y=df_substances['count'],
                        yaxis='y',
                        mode='lines',
                        marker=dict(
                            color='red'
                            ),
                        line=dict(
                            width=7
                            ),
                        opacity=1,
                        name='Testosterona',
                        line_shape='spline'
                        ),
                        secondary_y=False
    )

fig.add_trace(go.Scatter(
                        x=lf_gt['Mês'], 
                        y=lf_gt['impotência: (Brasil)'],
                        yaxis='y1',
                        mode='lines',
                        name='Google Trends: Impotência',
                        marker=dict(
                            symbol='53',
                            size=11

                        ),
                        line = dict(
                            color='blue',
                            width=5
                            ),
                        opacity=0.3,
                        line_shape='spline'
                        ),
                        secondary_y=True
    )

fig.add_trace(go.Scatter(
                        x=lf_hp['Mês'], 
                        y=lf_hp['Hipertrofia: (Brasil)'],
                        yaxis='y1',
                        mode='lines',
                        name='Google Trends: Hipertrofia',
                        marker=dict(
                            symbol='51',
                            size=11

                        ),
                        line = dict(
                            color='violet',
                            width=5
                            ),
                        opacity=0.3,
                        line_shape='spline'
                        ),
                        secondary_y=True
                        )


fig.update_layout(
    xaxis=dict(
        title='Data',
        titlefont=dict(
            size=30
        ),
        tickfont=dict(
            size=25
        )
    ),
    width=1500,
    height=1000,
    yaxis=dict(
        title="<b>Quantidade vendida<b>",
        titlefont=dict(
            color="red",
            size=30
            
        ),
        tickfont=dict(
            color="red",
            size=25
        )
    ),
    yaxis2=dict(
        title="Google trends (interesse no termo)",
        titlefont=dict(
            color="black",
            size=30
        ),
        tickfont=dict(
            color="black",
            size=25
        ),
        # anchor="free",
        overlaying="y",
        side="right",
        # position=0.15
    ))


fig.show()

## Analisando os anti-depressivos 